In [27]:
import pandas as pd


In [28]:
original_df = pd.read_csv("/dccstor/autofair/busekorkmaz/factual-bias-mitigation/data/output_files/hackernews_original_eval_df_with_facts.csv")
original_df.head(1)

,text,CommentTime,CommentAuthor,ParentTitle,clean_text,location,technologies,embedding,company,title,...,ir_asia,ir_remote,ir_australia,ir_africa,ir_sa,ir_unknown,ir_female,ir_male,q_val,atomic_facts
0,"Enchanted Labs - (San Francisco, CA) - Ex-Face...",2014-10-03 20:53:43 UTC,JuliaLam,Ask HN: Who is hiring? (October 2014),"Enchanted Labs - (San Francisco, CA) - Ex-Face...","San Francisco, CA",\nROLE\n- Back End/Fullstack Engineer,[-5.89705348e-01 4.93332267e-01 -3.13929349e-...,Enchanted Labs,Back End/Fullstack Engineer,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.772727,-26.852616,"['Enchanted Labs is located in San Francisco, ..."


In [18]:
generated_df = pd.read_csv("/dccstor/autofair/busekorkmaz/factual-bias-mitigation/data/output_files/hackernews_generated_eval_df_with_facts.csv")
generated_df.head(1)

,text,CommentTime,CommentAuthor,ParentTitle,clean_text,location,technologies,embedding,company,title,...,ir_africa,ir_sa,ir_unknown,ir_female,ir_male,q_val,diff_ir_female,diff_ir_male,diff_q_df,atomic_facts
0,"Enchanted Labs - (San Francisco, CA) - Ex-Face...",2014-10-03 20:53:43 UTC,JuliaLam,Ask HN: Who is hiring? (October 2014),"Enchanted Labs - (San Francisco, CA) - Ex-Face...","San Francisco, CA",\nROLE\n- Back End/Fullstack Engineer,[-5.89705348e-01 4.93332267e-01 -3.13929349e-...,Enchanted Labs,Back End/Fullstack Engineer,...,0.0,0.0,0.0,1.0,1.0,-11.352616,0.0,0.227273,15.5,"['Enchanted Labs is located in San Francisco, ..."


In [19]:
from IPython.display import display, HTML
display(HTML(original_df.loc[0]['text']))

In [20]:
factoids_df = pd.DataFrame(data = [], columns = ['original_text', 'generated_text', 'original_facts', 'generated_facts'])

In [21]:
factoids_df['original_text'] = original_df['evaluated_text']
factoids_df['generated_text'] = generated_df['evaluated_text']
factoids_df['original_facts'] = original_df['atomic_facts'].str.replace('[', '')
factoids_df['original_facts'] = factoids_df['original_facts'].str.replace(']', '').str.split(',')
factoids_df['original_facts'] = factoids_df['original_facts'].apply(lambda x: [y.strip("'") if isinstance(y, str) else y for y in x])
factoids_df['original_facts'] = factoids_df['original_facts'].apply(lambda x: [y.strip(" '") if isinstance(y, str) else y for y in x])
factoids_df['generated_facts'] = generated_df['atomic_facts'].str.replace('[', '')
factoids_df['generated_facts'] = factoids_df['generated_facts'].str.replace(']', '').str.split(',')
factoids_df['generated_facts'] = factoids_df['generated_facts'].apply(lambda x: [y.strip("'") if isinstance(y, str) else y for y in x])
factoids_df['generated_facts'] = factoids_df['generated_facts'].apply(lambda x: [y.strip(" '") if isinstance(y, str) else y for y in x])

factoids_df.head(5)

,original_text,generated_text,original_facts,generated_facts
0,"Enchanted Labs - (San Francisco, CA) - Ex-Face...","Enchanted Labs - (San Francisco, CA) - Ex-Face...","[Enchanted Labs is located in San Francisco, C...","[Enchanted Labs is located in San Francisco, C..."
1,"Euclid Inc - San Francisco's Mission District,...","Euclid Inc - San Francisco's Mission District,...","[""Euclid Inc is located in San Francisco's Mis...","[""Euclid Inc is located in San Francisco's Mis..."
2,"AppHero (apphero.com) - Toronto, Ontario, Cana...","AppHero (apphero.com) - Toronto, Ontario, Cana...","[AppHero is a company., AppHero is located in ...","[AppHero is a company., ""AppHero's website is ..."
3,Toronto - Uken Games\nSoftware Engineer (full ...,Toronto - Uken Games\nSoftware Engineer (full ...,[Uken Games is looking for talented software e...,[Uken Games is looking for talented software e...
4,Toronto - Uken Games\nSoftware Engineer (full ...,Toronto - Uken Games\nSoftware Engineer (full ...,[Uken Games is looking for talented software e...,[Uken Games is looking for talented software e...


In [22]:
def avg_length_of_factoids(factoids_df):
    generated_length = 0
    for row in factoids_df['generated_facts']:
        generated_length += len(row)
    
    original_length = 0
    for row in factoids_df['original_facts']:
        original_length += len(row)

    print("Original: ", original_length/len(factoids_df), "Generated: ", generated_length/len(factoids_df))    

In [23]:
avg_length_of_factoids(factoids_df)

Original:  27.805 Generated:  21.7525


In [24]:
factoids_df.loc[0]['original_facts'][0]

'Enchanted Labs is located in San Francisco'

In [25]:
factoids_df.loc[0]['generated_facts'][0]

'Enchanted Labs is located in San Francisco'

In [26]:
def remove_duplicate_facts(row):
    original = set(fact.lower().replace(',', '').replace('.', '').strip() for fact in row['original_facts'])
    generated = set(fact.lower().replace(',', '').replace('.', '').strip() for fact in row['generated_facts'])
    
    unique_original = [fact for fact in row['original_facts'] if fact.lower().replace(',', '').replace('.', '').strip() not in generated]
    unique_generated = [fact for fact in row['generated_facts'] if fact.lower().replace(',', '').replace('.', '').strip() not in original]
    
    return pd.Series({
        'original_facts': unique_original,
        'generated_facts': unique_generated
    })

# Apply the function to each row
factoids_df[['original_facts', 'generated_facts']] = factoids_df.apply(remove_duplicate_facts, axis=1)
avg_length_of_factoids(factoids_df)

Original:  17.4675 Generated:  11.3875


In [13]:
factoids_df.head()

,original_text,generated_text,original_facts,generated_facts
0,"Enchanted Labs - (San Francisco, CA) - Ex-Face...","Enchanted Labs - (San Francisco, CA) - Ex-Face...","[Enchanted Labs is located in San Francisco, E...",[Enchanted Labs is seeking individuals who can...
1,"Euclid Inc - San Francisco's Mission District,...","Euclid Inc - San Francisco's Mission District,...",[There are four full-time positions available ...,[There are four full-time positions available ...
2,"AppHero (apphero.com) - Toronto, Ontario, Cana...","AppHero (apphero.com) - Toronto, Ontario, Cana...","[AppHero is a company., AppHero wants to revol...","[The team is at AppHero., The engineer will he..."
3,Toronto - Uken Games\nSoftware Engineer (full ...,Toronto - Uken Games\nSoftware Engineer (full ...,[The software engineers will work on building ...,[The software engineers will be members of the...
4,Toronto - Uken Games\nSoftware Engineer (full ...,Toronto - Uken Games\nSoftware Engineer (full ...,[The software engineers will work on building ...,[The software engineers will be members of the...


In [14]:
factoids_df.loc[0]['original_facts']

['Enchanted Labs is located in San Francisco',
 'Enchanted Labs requires problem-solving skills from their team members.',
 'Enchanted Labs requires rapid learning from their team members.',
 'Enchanted Labs is offering an awesome startup journey.',
 'There is a Back End/Fullstack Engineer role available at Enchanted Labs.',
 'More information about Enchanted Labs can be found at http://jobs.enchantedlabs.com.',
 'It is possible to contact Enchanted Labs at REDACTED_EMAIL.']

In [15]:
factoids_df.loc[0]['generated_facts']

['Enchanted Labs is seeking individuals who can problem solve.',
 'Enchanted Labs is seeking individuals who can learn rapidly.',
 'Enchanted Labs is seeking individuals who are excited for an awesome startup journey.',
 'The job is located in San Francisco',
 'The company is Enchanted Labs.',
 'The position is Back End/Fullstack Engineer.',
 'More information about the job can be found at http://jobs.enchantedlabs.com.',
 'The job posting can also be contacted via email at REDACTED_EMAIL.']

In [60]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

# Load a pre-trained sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

def get_embeddings(facts):
    return model.encode(facts)

def create_similarity_matrix(row):
    original_facts = row['original_facts']
    generated_facts = row['generated_facts']
    
    # Get embeddings
    original_embeddings = get_embeddings(original_facts)
    generated_embeddings = get_embeddings(generated_facts)
    
    # Calculate similarity matrix
    similarity_matrix = cosine_similarity(original_embeddings, generated_embeddings)
    
    return similarity_matrix

# Apply the function to each row
factoids_df['similarity_matrix'] = factoids_df.apply(create_similarity_matrix, axis=1)

# Function to format the similarity matrix for display
def format_similarity_matrix(row):
    matrix = row['similarity_matrix']
    original_facts = row['original_facts']
    generated_facts = row['generated_facts']
    
    df = pd.DataFrame(matrix, 
                      index=[f"Original {i+1}" for i in range(len(original_facts))],
                      columns=[f"Generated {i+1}" for i in range(len(generated_facts))])
    return df

# Display the similarity matrix for the first row (you can change the index as needed)
print(format_similarity_matrix(factoids_df.iloc[0]))

# If you want to save all matrices for later analysis
factoids_df['formatted_similarity_matrix'] = factoids_df.apply(format_similarity_matrix, axis=1)

/u/busekorkmaz/.conda/envs/factuality/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/u/busekorkmaz/.conda/envs/factuality/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


ImportError: cannot import name 'insecure_hashlib' from 'huggingface_hub.utils' (/u/busekorkmaz/.conda/envs/factuality/lib/python3.10/site-packages/huggingface_hub/utils/__init__.py)

In [ ]:
similarity_matrix = factoids_df.iloc[0]['similarity_matrix']
high_similarity_pairs = np.where(similarity_matrix > 0.8)
for orig, gen in zip(*high_similarity_pairs):
    print(f"Original fact {orig+1} is highly similar to Generated fact {gen+1}")
    print(f"Similarity: {similarity_matrix[orig, gen]:.2f}")
    print(f"Original: {factoids_df.iloc[0]['original_facts'][orig]}")
    print(f"Generated: {factoids_df.iloc[0]['generated_facts'][gen]}")
    print()

In [ ]:
# from sentence_transformers import SentenceTransformer
# from sklearn.metrics.pairwise import cosine_similarity
# import numpy as np

# # Load a pre-trained sentence transformer model
# model = SentenceTransformer('all-MiniLM-L6-v2')

# def get_embeddings(facts):
#     return model.encode(facts)

# def remove_similar_facts(row, similarity_threshold=0.8):
#     original_facts = row['original_facts']
#     generated_facts = row['generated_facts']
    
#     # Get embeddings
#     original_embeddings = get_embeddings(original_facts)
#     generated_embeddings = get_embeddings(generated_facts)
    
#     # Calculate similarity matrix
#     similarity_matrix = cosine_similarity(original_embeddings, generated_embeddings)
    
#     # Find pairs of similar facts
#     similar_pairs = np.where(similarity_matrix > similarity_threshold)
    
#     # Create sets of indices to remove
#     original_to_remove = set(similar_pairs[0])
#     generated_to_remove = set(similar_pairs[1])
    
#     # Keep only dissimilar facts
#     unique_original = [fact for i, fact in enumerate(original_facts) if i not in original_to_remove]
#     unique_generated = [fact for i, fact in enumerate(generated_facts) if i not in generated_to_remove]
    
#     return pd.Series({
#         'original_facts': unique_original,
#         'generated_facts': unique_generated
#     })

# # Apply the function to each row
# factoids_df[['original_facts', 'generated_facts']] = factoids_df.apply(remove_similar_facts, axis=1)